<a href="https://colab.research.google.com/github/HKang42/DS-Unit-1-Build/blob/master/Harrison_Kang_LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

# Let's look at the data

In [7]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [8]:
# there's a ton of Nan values and x values in the later columns
# We need to do something about all of the NaN values and x values
# let's see how many nulls we have

df.isnull().sum()

Burrito             0
Date                0
Yelp              334
Google            334
Chips             395
Cost                7
Hunger              3
Mass (g)          399
Density (g/mL)    399
Length            138
Circum            140
Volume            140
Tortilla            0
Temp               20
Meat               14
Fillings            3
Meat:filling        9
Uniformity          2
Salsa              25
Synergy             2
Wrap                3
Unreliable        388
NonSD             414
Beef              242
Pico              263
Guac              267
Cheese            262
Fries             294
Sour cream        329
Pork              370
Chicken           400
Shrimp            400
Fish              415
Rice              385
Beans             386
Lettuce           410
Tomato            414
Bell peper        414
Carrots           420
Cabbage           413
Sauce             383
Salsa.1           414
Cilantro          406
Onion             404
Taquito           417
Pineapple 

In [9]:
# That looks pretty bad. But we know that a lot of these null values just mean the burrito lacked the ingredient.
# Use the replace function to fix all of the x's and Nan's from the Unreliable to the Zucchini columns
# for the legitimate null values, we can just impute them later

import numpy as np

df.loc[:,'Unreliable':'Zucchini'] = df.loc[:,'Unreliable':'Zucchini'].replace( {np.NaN : 0 , 'x' : 1, 'X':1})
df.loc[:,'Unreliable':'Zucchini']

,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini
0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
1,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
3,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
4,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
419,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
420,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0
421,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0


In [0]:
# we also need to do the same to the chips column

# df['Chips'].value_counts()

df['Chips'] = df['Chips'].replace( {np.NaN : 0 , 'x' : 1, 'X':1 , 'Yes':1, 'No' :0})

In [22]:
# Let's look at some value counts now
df['Unreliable'].value_counts(dropna = False)

#df.iloc[:,8].value_counts(dropna = False)

0    388
1     33
Name: Unreliable, dtype: int64

# Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.


In [0]:
# convert Date strings to datetime format so pandas can sort them properly
df['Date'] = pd.to_datetime(df['Date'])

# convert the Great column to numerical values
df['Great'] = df['Great'].replace({True: 1, False: 0})

In [24]:
# now we split into train, validate, and test
train = df[df['Date'] < '2017']
val = df[ (df['Date'] >= '01-01-2017' ) & (df['Date'] <= '01-01-2018')]
test = df[df['Date'] >= '2018']



# use to make sure things were split properly
"""
sorted_dates = train['Date'].value_counts().index.sort_values()

for date in s:
  print(date)

"""

"\nsorted_dates = train['Date'].value_counts().index.sort_values()\n\nfor date in s:\n  print(date)\n\n"

In [0]:
# let's split train based on target and independent variables

target = 'Great'

train_x = train.iloc[:, :-1]
train_y = train[target]

val_x = val.iloc[:, :-1]
val_y = val[target]


test_x = test.iloc[:, :-1]
test_y = test[target]

#  Begin with baselines for classification.

In [26]:
# let's see what's the most common value

train_y.value_counts(normalize=True)

0    0.590604
1    0.409396
Name: Great, dtype: float64

In [0]:
# assign the majority class to the most common vale
majority_class = train_y.mode()

# create a dataframe that predicts the majority class for every row.  This is our baseline prediction
y_pred = [majority_class] * len(train_x)

In [28]:
# Let's print our accuracy score so we can use it for comparison later
from sklearn.metrics import accuracy_score

print('Baseline accuracy score:', accuracy_score(train_y, y_pred).round(4) )

Baseline accuracy score: 0.591


#  Use scikit-learn for logistic regression.

In [43]:
# Let's drop the location column for now.  We can one-hot encode it later if we want.

target = 'Great'

train_x = train.iloc[:, :-1]
train_y = train[target]

val_x = val.iloc[:, :-1]
val_y = val[target]


test_x = test.iloc[:, :-1]
test_y = test[target]


train_x = train_x.drop( columns=['Burrito', 'Date', 'Queso' ])
val_x = val_x.drop(columns=['Burrito', 'Date', 'Queso'])
test_x = test_x.drop(columns=['Burrito', 'Date', 'Queso'])

"""
train_x = train_x.loc[:, 'Yelp' : 'Cost']
val_x = val_x.loc[:, 'Yelp' : 'Cost']
test_x = test_x.loc[:, 'Yelp' : 'Cost']

train_x = train_x.loc[:, ['Pineapple' , 'Lobster']]
val_x = val_x.loc[:, ['Pineapple' , 'Lobster']]
test_x = test_x.loc[:, ['Pineapple' , 'Lobster']]
"""

"\ntrain_x = train_x.loc[:, 'Yelp' : 'Cost']\nval_x = val_x.loc[:, 'Yelp' : 'Cost']\ntest_x = test_x.loc[:, 'Yelp' : 'Cost']\n\ntrain_x = train_x.loc[:, ['Pineapple' , 'Lobster']]\nval_x = val_x.loc[:, ['Pineapple' , 'Lobster']]\ntest_x = test_x.loc[:, ['Pineapple' , 'Lobster']]\n"

In [0]:
from sklearn.impute import SimpleImputer

# instantiate our imputer
imputer = SimpleImputer( strategy='mean')

# use imputer on our training and validation sets
train_x_imputed = imputer.fit_transform(train_x)

val_x_imputed = imputer.transform(val_x)

test_x_imputed = imputer.transform(test_x)

In [45]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression( solver='lbfgs' , max_iter=1000)

log_reg.fit( train_x_imputed ,train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#  Get your model's validation accuracy. (Multiple times if you try multiple iterations.)

In [46]:
print('Validation Accuracy', log_reg.score(val_x_imputed, val_y).round(4) )

print('\nFeatures used are\n\n', train_x.columns)

Validation Accuracy 0.8

Features used are

 Index(['Yelp', 'Google', 'Chips', 'Cost', 'Hunger', 'Mass (g)',
       'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla', 'Temp',
       'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa', 'Synergy',
       'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac', 'Cheese',
       'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish', 'Rice',
       'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots', 'Cabbage',
       'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito', 'Pineapple', 'Ham',
       'Chile relleno', 'Nopales', 'Lobster', 'Egg', 'Mushroom', 'Bacon',
       'Sushi', 'Avocado', 'Corn', 'Zucchini'],
      dtype='object')


That's a pretty good improvement over our baseline's accuracy of 59%.

# Get your model's test accuracy. (One time, at the end.)

In [51]:
print('Validation Accuracy', log_reg.score(test_x_imputed, test_y).round(4) )

print('\nFeatures used are\n\n', train_x.columns)

Validation Accuracy 0.7895

Features used are

 Index(['Yelp', 'Google', 'Chips', 'Cost', 'Hunger', 'Mass (g)',
       'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla', 'Temp',
       'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa', 'Synergy',
       'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac', 'Cheese',
       'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish', 'Rice',
       'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots', 'Cabbage',
       'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito', 'Pineapple', 'Ham',
       'Chile relleno', 'Nopales', 'Lobster', 'Egg', 'Mushroom', 'Bacon',
       'Sushi', 'Avocado', 'Corn', 'Zucchini'],
      dtype='object')


# Code below not part of assignment

In [47]:
df.groupby('Burrito').mean()

,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
Burrito,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Asada,3.857143,4.214286,0.023256,6.514419,3.469767,435.000,0.621324,19.948276,21.292857,0.727500,3.470930,3.600000,3.383333,3.358140,3.744186,3.546512,3.398810,3.363953,4.000000,0.046512,0.046512,0.720930,0.651163,0.697674,0.116279,0.000000,0.093023,0.000000,0.023256,0.000000,0.000000,0.069767,0.093023,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.023256,0.023256,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.418605
California,3.845455,4.111364,0.071006,7.008750,3.588095,544.500,0.706291,20.195714,22.285805,0.800678,3.609763,3.829560,3.656213,3.675444,3.589941,3.429167,3.467178,3.752395,4.269231,0.041420,0.000000,0.579882,0.414201,0.343195,0.603550,0.621302,0.396450,0.017751,0.029586,0.005917,0.000000,0.011834,0.000000,0.005917,0.011834,0.000000,0.00000,0.000000,0.017751,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.005917,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.508876
Carnitas,3.750000,4.100000,0.040000,6.237200,3.438000,432.500,0.697396,19.368421,21.500000,0.713684,3.204000,3.740000,3.952000,3.472000,3.994000,3.292000,3.113043,3.508000,3.916667,0.040000,0.000000,0.000000,0.680000,0.680000,0.040000,0.000000,0.080000,0.760000,0.000000,0.000000,0.000000,0.080000,0.040000,0.000000,0.000000,0.040000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.00000,0.440000
Other,3.983333,4.256667,0.064103,7.226400,3.486452,594.375,0.659561,19.954388,22.081378,0.782143,3.487821,3.855782,3.565385,3.471429,3.434324,3.402258,3.282042,3.508654,3.692857,0.134615,0.032051,0.198718,0.192308,0.282051,0.262821,0.089744,0.115385,0.185897,0.096154,0.012821,0.038462,0.128205,0.185897,0.064103,0.032051,0.025641,0.00641,0.044872,0.134615,0.044872,0.089744,0.096154,0.025641,0.044872,0.012821,0.019231,0.025641,0.00641,0.0,0.025641,0.019231,0.019231,0.012821,0.006410,0.012821,0.00641,0.371795
Surf & Turf,3.750000,4.050000,0.035714,8.157143,3.062963,585.000,0.645924,20.305556,23.430556,0.885556,3.507143,3.450000,3.744643,3.433333,3.770370,3.517857,3.451923,3.450000,3.835714,0.071429,0.000000,0.678571,0.464286,0.178571,0.357143,0.285714,0.035714,0.000000,0.000000,0.642857,0.000000,0.321429,0.035714,0.000000,0.000000,0.071429,0.00000,0.035714,0.500000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.428571,0.000000,0.00000,0.321429


In [38]:
df.loc[:, ['Burrito', 'Great']].groupby('Burrito').mean()

,Great
Burrito,
Asada,0.418605
California,0.508876
Carnitas,0.440000
Other,0.371795
Surf & Turf,0.321429
